In [1]:
import Profit_Test_Class as pt
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import statistics as stat

from matplotlib.font_manager import FontProperties
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.cluster import AgglomerativeClustering
from sklearn import manifold
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from abc import ABCMeta, abstractmethod

%matplotlib inline
plt.rcParams['axes.unicode_minus']=False
engine = create_engine('sqlite:///fund.db')

# 樣板

In [2]:

class (pt.ProfitVisualize):
    
    '''
    input:
        NAV:一個月的淨值
        names:與淨值對齊的名字
    return:
        features:找出的features
        names:與features對齊的名字
        距離與否：是距離的話填'precomputed'，不是的話填'euclidean'
    '''       
    def getFeatures(self, NAV, names):
        
        return features,names,'precomputed'
    
    '''
    input:
        year:回測年
    return:
        names:選出來進入分群階段的基金名字
    '''    
    def poolDecide(self,year):
        
        return names

SyntaxError: invalid syntax (<ipython-input-2-8ca5f347c96b>, line 2)

# 範例

In [3]:
'''
每日漲跌幅相關係數
'''
class UpDownRateSimilarity(pt.ProfitVisualize):
        
    def getFeatures(self, NAV, names):
        length = len(NAV[0])-1
        rate = np.zeros((len(names),length))
        for j in range (len(names)):
            for i in range (length):
                rate[j][i] = (NAV[j][i+1] - NAV[j][i]) / NAV[j][i]
        
        temp = []
        for i,j in enumerate(rate):                        
            if np.cov(j) == 0:
                temp.append(i)                
        rate = np.delete(rate,temp,0)
        names = np.delete(names,temp,0)
        
        similarity = np.zeros((len(rate),len(rate)))
        for i in range(len(rate)):
            for j in range(len(rate)):
                corr = np.corrcoef(rate[i],rate[j])[0][-1]
                if i == j:
                    corr = 1
                similarity[i][j] = 1-(corr*0.5+0.5)
                
        return similarity,names,'precomputed'
    
    def poolDecide(self,year):
        names = pd.read_sql(sql='select distinct id from price where date between ? and ? order by date asc',
                                 con=engine, params=[year+'-01-01',year+'-01-31'])
        names = names['id'].sample(n=300).values
        return names

In [ ]:
profitVisualize = UpDownRateSimilarity(pt.K_Means)
profitVisualize.getProfitPicture()